In [49]:
import pandas as pd
import MeCab as mc
from wordcloud import WordCloud
import os
import re
from wordcloud import STOPWORDS

# Twitterの外部ライブラリ(GOT3)の使い方
python3 cli.py -s (検索ワード) --since "2022-12-05"(検索開始日時) --untill "2022-12-06"(検索終了日時) -o user.csv(ファイル名) --csv

In [19]:
def reduceTwitterData(rawData):
    df = pd.read_csv("../GetOldTweets3-0.0.10/" + rawData)
    print(df)
    df.to_csv('./csv_Data/revised_'+ rawData, columns = ['date', 'time', 'timezone', 'tweet'], index = False)

In [3]:
def strip_CRLF_from_Text(text):
    """
    改行前後が日本語文字の場合は改行を削除する．
    それ以外はスペースに置換する．
    """
    # 改行前後の文字が日本語文字の場合は改行を削除する
    plaintext = re.sub('([ぁ-んー]+|[ァ-ンー]+|[\\u4e00-\\u9FFF]+|[ぁ-んァ-ンー\\u4e00-\\u9FFF]+)(\n)([ぁ-んー]+|[ァ-ンー]+|[\\u4e00-\\u9FFF]+|[ぁ-んァ-ンー\\u4e00-\\u9FFF]+)',
                       r'\1\3',
                       text)
    plaintext = re.sub('http(s)?://([\w]+\.)+[\w]+(/[\w ./?%&=]*)?', "", plaintext)
    # 残った改行とタブ記号はスペースに置換する
    plaintext = plaintext.replace('\n', ' ').replace('\t', ' ')
    return plaintext

In [4]:
def mecab_wakati(text):
    #t = mc.Tagger()
    t = mc.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
    node = t.parseToNode(text)
    #print(type(node))
    sent = ""
    while(node):
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            # 名詞だけをリストに追加する
            # if word_type in ["名詞"] and (("日本") not in node.surface) and (("代表") not in node.surface) and (("選手") not in node.surface) and (("監督") not in node.surface) and (("試合") not in node.surface):
            #      sent += node.surface + " "  # node.surface は「表層形」
            # 動詞（の原型），形容詞，副詞もリストに加えたい場合は次の２行を有効にする
            if word_type in [ "形容動詞", "形容詞"]:
                sent += node.feature.split(",")[6] + " " 
        node = node.next
        if node is None:
            break
    return sent

In [103]:
def twitterdataToWordCloud(CSV_FILE):
    df = pd.read_csv("./csv_data/revised_" + CSV_FILE, engine="python")
    tweet = df.loc[: , "tweet"]
    tweetcontent = ""
    for i in range(len(tweet)):
        tweetcontent += str(tweet[i])
    # f = open(tweetcontent, encoding="utf8")
    # raw = f.read()
    #print(raw)
    text = strip_CRLF_from_Text(tweetcontent)
    #print(text)
    sent = mecab_wakati(text)
    sent = re.sub('(\s)+(.)(\s)+', ' ', sent)
    # フォントの保存先を指定する
    font_path = "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc" 

    STOPWORDS.add("ない")
    wc = WordCloud(max_font_size=120, font_path=font_path, background_color="white", colormap="winter", width=800, height=400).generate(sent)
    wc.to_file("./wordCloud/wc-" + CSV_FILE[:-3] + "png")

In [14]:
def main(rawdata):  #nation.csv
    reduceTwitterData(rawdata)
    twitterdataToWordCloud(rawdata)

In [158]:
main("韓国-2022.csv")

                       id      conversation_id               created_at  \
0     1598693412022419456  1598693412022419456  2022-12-02 23:59:46 JST   
1     1598693396842848256  1598693396842848256  2022-12-02 23:59:42 JST   
2     1598693374516776961  1598693374516776961  2022-12-02 23:59:37 JST   
3     1598693348629520385  1598693348629520385  2022-12-02 23:59:31 JST   
4     1598693312252502017  1598693312252502017  2022-12-02 23:59:22 JST   
...                   ...                  ...                      ...   
8742  1594136958300667906  1594136958300667906  2022-11-20 10:14:03 JST   
8743  1594134698002812930  1594134698002812930  2022-11-20 10:05:04 JST   
8744  1594124092718067713  1594124092718067713  2022-11-20 09:22:55 JST   
8745  1594123356516786178  1594123356516786178  2022-11-20 09:20:00 JST   
8746  1594120931974135808  1594118505640603648  2022-11-20 09:10:22 JST   

            date      time  timezone              user_id         username  \
0     2022-12-02  23:

In [44]:
twitterdataToWordCloud("セネガル-2022.csv")